# Setup

In [ ]:
!sudo apt-get install sumo sumo-tools sumo-doc

In [ ]:
!pip install qiskit[visualization]

In [ ]:
!pip install dwave-qiskit-plugin

In [ ]:
!dwave setup

# Start Here

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import vrp_xml_handler
import quantum_optimizer
from itertools import product

In [2]:
n = 4
k = 2

In [3]:
node_dict = {0: '-168940071#1', 1: '4610479#0', 2: '234463931#2', 3: '171826984#2'}
edge_list = [p for p in product(node_dict.keys(), node_dict.keys()) if p[0] != p[1]]
path_dict = {i: edge_list[i] for i in range(len(edge_list))}
depart_times = range(0, 3600, 300)
departure_schedule = {t: list(path_dict.keys()) for t in reversed(depart_times)}
vrp_xml_handler.build_simulation_route(node_dict, path_dict, departure_schedule, 'scenario_1')

In [4]:
os.system('sumo -c scenario_1/osm.sumocfg --tripinfo-output scenario_1/vrp_cost_sim.xml --no-warnings true')

0

In [5]:
w = vrp_xml_handler.estimate_cost_graph('scenario_1/vrp_cost_sim.xml', path_dict)

In [6]:
instance = np.zeros((n, n))
for i, j in edge_list:
    instance[i, j] = w[(i, j)]

In [7]:
quantum_optimizer = quantum_optimizer.QuantumOptimizer(instance, n, k)
Q, g, c, binary_cost = quantum_optimizer.binary_representation()
qp = quantum_optimizer.construct_problem(Q, g, c)
quantum_solution, quantum_cost = quantum_optimizer.solve_problem(qp)

In [8]:
route_edges = [edge_list[i] for i in range(len(edge_list)) if quantum_solution[i] == 1]
routes = [list(edge) for edge in route_edges if edge[0] == 0]
for route in routes:
    while route[-1] != 0:
        stuck = True
        for edge in route_edges:
            if edge[0] == route[-1]:
                route.append(edge[1])
                stuck = False
                break
        if stuck:
            break

In [9]:
routes = [tuple(route) for route in routes]
path_dict = {i: routes[i] for i in range(len(routes))}
depart_times = range(0, 3600, 1200)
schedule = {t: list(path_dict.keys()) for t in reversed(depart_times)}
vrp_xml_handler.build_simulation_route(node_dict, path_dict, schedule)
print('Initiating Full Simulation...')
os.system('sumo -c {0}/osm.sumocfg --tripinfo-output {0}/vrp_full_sim.xml --no-warnings true'.format('scenario_1'))
print('Optimized Quantum Cost: ', quantum_cost)

Initiating Full Simulation...
Optimized Quantum Cost:  1011.4999999997672
